In [2]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import RegexTokenizer, StopWordsRemover, CountVectorizer
from pyspark.ml.classification import LogisticRegression, LogisticRegressionModel
from pyspark.ml import Pipeline
from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorAssembler
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
import os

In [3]:
mongo_url = '192.168.1.11'
mongo_port = '27017'
mongo_username = 'root'
mongo_password = 'passw0rd'

mongo_db_url = "mongodb://" + \
               mongo_username + \
               ":" + mongo_password + \
               "@" + mongo_url + \
               ":" + str(mongo_port) + \
               "/news_feed.raw_news?authSource=admin"

In [4]:
exampleDir = os.path.join("/Users/admin/Documents/personal/mentor/IIIT-HyderabadCourse/Capstone-NewsClassifier/model-trainer", "resources/jars")
exampleJars = [os.path.join(exampleDir, x) for x in os.listdir(exampleDir)]

In [5]:
exampleJars

['/Users/admin/Documents/personal/mentor/IIIT-HyderabadCourse/Capstone-NewsClassifier/model-trainer/resources/jars/mongo-spark-connector_2.12-3.0.1.jar']

In [6]:
spark = SparkSession.builder.master("local[*]").appName('PySpark_tutorial') \
    .config("spark.mongodb.input.uri", mongo_db_url) \
    .config("spark.mongodb.output.uri", mongo_db_url) \
    .config("spark.jars", ",".join(exampleJars))\
    .getOrCreate()

In [7]:
df = spark.read.format("com.mongodb.spark.sql.DefaultSource").load()

Py4JJavaError: An error occurred while calling o43.load.
: java.lang.NoClassDefFoundError: org/bson/conversions/Bson
	at com.mongodb.spark.sql.DefaultSource.constructRelation(DefaultSource.scala:89)
	at com.mongodb.spark.sql.DefaultSource.createRelation(DefaultSource.scala:50)
	at org.apache.spark.sql.execution.datasources.DataSource.resolveRelation(DataSource.scala:355)
	at org.apache.spark.sql.DataFrameReader.loadV1Source(DataFrameReader.scala:325)
	at org.apache.spark.sql.DataFrameReader.$anonfun$load$3(DataFrameReader.scala:307)
	at scala.Option.getOrElse(Option.scala:189)
	at org.apache.spark.sql.DataFrameReader.load(DataFrameReader.scala:307)
	at org.apache.spark.sql.DataFrameReader.load(DataFrameReader.scala:225)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invok

In [1]:
df.printSchema()

NameError: name 'df' is not defined

In [10]:
df.head(4)

[Row(_id=Row(oid='615fb3658be95e04bd94883f'), author='Matt Ferner', link='https://www.huffingtonpost.com/entry/ap-history-protest_n_5890768.html', published_date='2014-09-27T00:00:00', summary='', title='Students Protesting Conservative Rewrite Of History Get Big Endorsement', topic='POLITICS'),
 Row(_id=Row(oid='615fb3668be95e04bd948840'), author='David Lohr', link='https://www.huffingtonpost.com/entry/nebraska-missing-woman-persons-of-interest_us_5a21cacfe4b0a02abe9135c4', published_date='2017-12-02T00:00:00', summary='The pair were taken into custody after posting a bizarre series of videos online.', title="Mother Of Woman Missing After Tinder Date Hopeful After 'Persons Of Interest' Arrested", topic='QUEER VOICES'),
 Row(_id=Row(oid='615fb3708be95e04bd948841'), author='Sarah St. Lifer', link='https://www.huffingtonpost.com/entry/diane-von-furstenberg-fall-2012_us_5b9b51e7e4b03a1dcc7697d7', published_date='2012-02-13T00:00:00', summary='Who Was There: Anderson Cooper, Barbara Walter

In [11]:
drop_list = ['author', 'link', 'published_date', 'title']
data = df.select([column for column in df.columns if column not in drop_list])

In [12]:
data.head(1)

[Row(_id=Row(oid='615fb3658be95e04bd94883f'), summary='', topic='POLITICS')]